
# Interpretabilidad de Modelos de Machine Learning

La interpretabilidad es la capacidad de entender por qué un modelo toma una decisión determinada. Es clave para:

- Tomar decisiones informadas.
- Garantizar la equidad.
- Detectar errores.
- Cumplir con regulaciones.

---

## Objetivos del notebook:

1. Comprender distintas técnicas de interpretabilidad.
2. Aplicar métodos globales y locales.
3. Analizar la importancia de características, contribuciones individuales, y visualizaciones.
4. Introducir `SHAP`, `Permutation Importance`, `Partial Dependence`, `ALE`, y más.


In [ ]:

from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

# Cargar datos
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo = RandomForestRegressor(random_state=42)
modelo.fit(X_train, y_train)



## 1. Importancia de Características (Global)

Los modelos de árbol permiten calcular la **importancia global** de cada variable:

- Basada en reducción del error (impureza).
- Mide qué tanto mejora la predicción al usar una variable.

Sin embargo, puede estar sesgada hacia variables con muchos niveles o alta cardinalidad.


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

importancias = pd.Series(modelo.feature_importances_, index=X.columns).sort_values()
importancias.plot(kind='barh', figsize=(8, 5), title="Importancia Global de Características")
plt.grid(True)
plt.show()



## 2. Permutation Importance (Global)

Evalúa la importancia de una variable **perturbando** sus valores y midiendo la caída en desempeño.

- Más fiable que la importancia basada en árboles.
- No depende de la estructura del modelo.


In [ ]:

from sklearn.inspection import permutation_importance

result = permutation_importance(modelo, X_test, y_test, n_repeats=10, random_state=42)
perm_df = pd.Series(result.importances_mean, index=X.columns).sort_values()

perm_df.plot(kind='barh', figsize=(8, 5), title="Permutation Importance")
plt.grid(True)
plt.show()



## 3. Gráficos de Dependencia Parcial (PDP)

Visualizan el **efecto promedio** de una característica en la predicción.

- Buena para interpretabilidad global.
- Asume independencia entre variables.

```python
from sklearn.inspection import PartialDependenceDisplay
```


In [ ]:

from sklearn.inspection import PartialDependenceDisplay

fig, ax = plt.subplots(figsize=(8, 4))
PartialDependenceDisplay.from_estimator(modelo, X, features=["LSTAT"], ax=ax)
plt.show()



## 5. SHAP: SHapley Additive exPlanations

Descompone la predicción de cada observación en **contribuciones de cada característica**, usando teoría de juegos.

### Ventajas:

- Coherente y localmente exacto.
- Compatible con modelos arbitrarios (`KernelExplainer`, `TreeExplainer`).

```bash
pip install shap
```


In [ ]:

import shap

explainer = shap.Explainer(modelo, X_train)
shap_values = explainer(X_test)

# SHAP summary plot
shap.plots.beeswarm(shap_values)



## Ejercicio Final

1. Cambia el modelo a un `GradientBoostingRegressor` o `XGBoost`.
2. Aplica `PartialDependenceDisplay` y `SHAP` a dos variables simultáneamente.
3. Usa `shap.plots.waterfall()` para analizar un caso individual.

¿Puedes explicar qué variables impulsaron una predicción hacia arriba o hacia abajo?
